In [31]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import datetime as dt
from datetime import date, timedelta
import plotly.graph_objects as go
import plotly.express as px

from datetime import date
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX


In [32]:

date.today()


datetime.date(2024, 12, 13)

In [33]:
today = date.today()
d1 = today.strftime("%Y-%m-%d")
end_date = d1
d2 = date.today() - timedelta(days=365)
d2 = d2.strftime("%Y-%m-%d")
start_date = d2
print("Your data will be between these datas:",start_date, end_date)

Your data will be between these datas: 2023-12-14 2024-12-13


In [34]:
# define the ticker
ticker = 'GOOGL'

df = yf.download(ticker, start=start_date, end=end_date, progress=False)
df.head()


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,GOOGL,GOOGL,GOOGL,GOOGL,GOOGL,GOOGL
Date,,,,,,
2023-12-14,131.463440,131.940002,133.720001,129.690002,133.380005,38722400
2023-12-15,132.121048,132.600006,133.509995,131.179993,131.619995,50815200
2023-12-18,135.309494,135.800003,137.149994,132.429993,132.630005,32258000
2023-12-19,136.156418,136.649994,137.470001,136.080002,136.839996,25476800
2023-12-20,137.840302,138.339996,141.699997,138.070007,138.970001,49107200


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 251 entries, 2023-12-14 to 2024-12-12
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   (Adj Close, GOOGL)  251 non-null    float64
 1   (Close, GOOGL)      251 non-null    float64
 2   (High, GOOGL)       251 non-null    float64
 3   (Low, GOOGL)        251 non-null    float64
 4   (Open, GOOGL)       251 non-null    float64
 5   (Volume, GOOGL)     251 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 13.7 KB


In [36]:
df["Date"] = df.index
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 251 entries, 2023-12-14 to 2024-12-12
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   (Adj Close, GOOGL)  251 non-null    float64       
 1   (Close, GOOGL)      251 non-null    float64       
 2   (High, GOOGL)       251 non-null    float64       
 3   (Low, GOOGL)        251 non-null    float64       
 4   (Open, GOOGL)       251 non-null    float64       
 5   (Volume, GOOGL)     251 non-null    int64         
 6   (Date, )            251 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 15.7 KB


In [37]:
df["Date"] = df.index
df.reset_index(drop=True, inplace=True)


In [38]:
df = df[['Date','Open','High','Low','Close','Adj Close','Volume']]

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   (Date, )            251 non-null    datetime64[ns]
 1   (Open, GOOGL)       251 non-null    float64       
 2   (High, GOOGL)       251 non-null    float64       
 3   (Low, GOOGL)        251 non-null    float64       
 4   (Close, GOOGL)      251 non-null    float64       
 5   (Adj Close, GOOGL)  251 non-null    float64       
 6   (Volume, GOOGL)     251 non-null    int64         
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 13.9 KB


In [40]:
df.insert(0,'Date',df.index,True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   (Date, )            251 non-null    int64         
 1   (Date, )            251 non-null    datetime64[ns]
 2   (Open, GOOGL)       251 non-null    float64       
 3   (High, GOOGL)       251 non-null    float64       
 4   (Low, GOOGL)        251 non-null    float64       
 5   (Close, GOOGL)      251 non-null    float64       
 6   (Adj Close, GOOGL)  251 non-null    float64       
 7   (Volume, GOOGL)     251 non-null    int64         
dtypes: datetime64[ns](1), float64(5), int64(2)
memory usage: 15.8 KB


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   (Date, )            251 non-null    int64         
 1   (Date, )            251 non-null    datetime64[ns]
 2   (Open, GOOGL)       251 non-null    float64       
 3   (High, GOOGL)       251 non-null    float64       
 4   (Low, GOOGL)        251 non-null    float64       
 5   (Close, GOOGL)      251 non-null    float64       
 6   (Adj Close, GOOGL)  251 non-null    float64       
 7   (Volume, GOOGL)     251 non-null    int64         
dtypes: datetime64[ns](1), float64(5), int64(2)
memory usage: 15.8 KB


In [42]:
df.head()

Price  Date                   Open        High         Low       Close  \
Ticker                       GOOGL       GOOGL       GOOGL       GOOGL   
0         0 2023-12-14  133.380005  133.720001  129.690002  131.940002   
1         1 2023-12-15  131.619995  133.509995  131.179993  132.600006   
2         2 2023-12-18  132.630005  137.149994  132.429993  135.800003   
3         3 2023-12-19  136.839996  137.470001  136.080002  136.649994   
4         4 2023-12-20  138.970001  141.699997  138.070007  138.339996   

Price    Adj Close    Volume  
Ticker       GOOGL     GOOGL  
0       131.463440  38722400  
1       132.121048  50815200  
2       135.309494  32258000  
3       136.156418  25476800  
4       137.840302  49107200

In [43]:
import yfinance as yf

df = yf.download('AAPL', start='2020-01-01', end='2024-01-01')

df = df[['Close']]  
print(df.head())

[*********************100%***********************]  1 of 1 completed

Price           Close
Ticker           AAPL
Date                 
2020-01-02  75.087502
2020-01-03  74.357498
2020-01-06  74.949997
2020-01-07  74.597504
2020-01-08  75.797501


In [55]:
df.head()

Price,Date,Close
Ticker,,AAPL
0,2020-01-02,75.087502
1,2020-01-03,74.357498
2,2020-01-06,74.949997
3,2020-01-07,74.597504
4,2020-01-08,75.797501


In [56]:
print(df['Date'].shape)  # Should output something like (1006,) indicating a 1D array
print(df['Close'].shape)  # Same as above


(1006,)
(1006, 1)


In [62]:
import yfinance as yf
import plotly.express as px

# Fetch Google stock data from Yahoo Finance
ticker = 'GOOG'  # You can also use 'GOOGL' for Alphabet Inc. Class A shares
stock_data = yf.download(ticker, start="2020-01-01", end="2024-01-01")

# Adding a 50-day moving average
stock_data['SMA50'] = stock_data['Close'].rolling(window=50).mean()

# Reset the index to ensure 'Date' is a column
stock_data.reset_index(inplace=True)

# Ensure both 'Close' and 'SMA50' are 1-dimensional
stock_data['Close'] = stock_data['Close'].values
stock_data['SMA50'] = stock_data['SMA50'].values

fig = px.line(df, x='Date', y='Close',title='Google Stock Price')

fig.show()



*********************100%***********************]  1 of 1 completed

ValueError: Data must be 1-dimensional, got ndarray of shape (1006, 1) instead